# Exploración Inicial

__Descripción:__

Se obtiene el AGEB de origen y destino de los viajes de los viajes de ecobici y se obtiene el agregado primedio de los viajes entre AGEBS por hora.

__Input__

- Estaciones de ecobici: Estaciones de ecobici con clave AGEB obtenida de __Uber © 2019 Copyright Uber Technologies, Inc.Data Attributions__

- Viajes Ecobici 2019
Responsable: Ecobici



In [1]:
import pandas as pd
import pickle

import pandas as pd
import dask.dataframe as dd
import time

import scipy.stats

## Obtencion de AGEBS de interés:

In [2]:
df_estaciones = pd.read_csv("./SIG/data/procesado/CSV/estaciones-de-ecobici_con_AGEB.csv")
df_estaciones.head()

,ID,Nombre,Dirección,Número,Código po,districtCo,Colonia,altitude,nearbyStat,location/l,...,nearbySt_3,nearbySt_4,nearbySt_5,punto_geo,CVE_AGEB,CVE_MUN,CVE_LOC,CVE_ENT,MOVEMENT_I,DISPLAY_NA
0,440,440 REAL DE MAYORAZGO-PUENTE XOCO,440 - Real De Mayorazgo-Puente Xoco,S/N,3330.0,1,Ampliación Granada,NaN,433,19.360650,...,NaN,NaN,NaN,"19.36065,-99.168669",0967,14,1,9,4550,Movement Zone 4550
1,283,283 AV. COYOACÁN-RAFAEL DONDE,283 - Av. Coyoacán-Rafael Donde,S/N,3103.0,1,Ampliación Granada,NaN,279,19.398270,...,NaN,NaN,NaN,"19.39827,-99.167383",0045,14,1,9,4551,Movement Zone 4551
2,279,279 LA MORENA-PROVIDENCIA,279 - La Morena-Providencia,S/N,3103.0,DVN,Del Valle Norte,NaN,164,19.399908,...,NaN,NaN,NaN,"19.399908,-99.166553",0045,14,1,9,4551,Movement Zone 4551
3,443,443 BRUNO TRAVEN-GOLONDRINAS,443 - Bruno Trave-Avenida México-Coyoacan,S/N,3340.0,1,Ampliación Granada,NaN,432,19.359655,...,NaN,NaN,NaN,"19.359655,-99.162025",0971,14,1,9,4553,Movement Zone 4553
4,432,432 CARRILLO PUERTO-AV. MÉXICO-COYOACÁN,432 - Carrillo Pto-Golondrinas,S/N,3340.0,GPM,General Pedro Maria Anaya,NaN,442,19.361296,...,NaN,NaN,NaN,"19.361296,-99.162581",0971,14,1,9,4553,Movement Zone 4553


In [25]:
lista_agebs = df_estaciones['CVE_AGEB'].unique().tolist()
len(lista_agebs)

143

Guardamos lo AGEBS en una lista:

In [4]:
with open("./data/lista_agebs.txt", "wb") as fp:
    pickle.dump(lista_agebs, fp)

## Leamos el archivo de viajes de UBER

In [40]:
path_uber_trips = "./data/uber/mexico_city-agebs-2019-1-OnlyWeekdays-HourlyAggregate.csv"

In [48]:
t=time.clock()
df_trips = pd.read_csv(path_uber_trips)
print("load train: " , time.clock()-t)

load train:  32.30997000000002


In [49]:
df_trips.dtypes

sourceid                                      int64
dstid                                         int64
hod                                           int64
mean_travel_time                            float64
standard_deviation_travel_time              float64
geometric_mean_travel_time                  float64
geometric_standard_deviation_travel_time    float64
dtype: object

In [42]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,1599,1710,4,629.46,600.50,498.86,1.80
1,3277,3083,8,1651.38,672.78,1549.81,1.40
2,977,3047,6,1165.89,268.38,1137.46,1.25
3,963,3187,6,931.03,284.51,898.61,1.29
4,3758,4915,13,2632.10,510.63,2583.73,1.21


In [43]:
df_trips = df_trips[df_trips["sourceid"].astype(str).isin(lista_agebs) & df_trips["dstid"].astype(str).isin(lista_agebs)]

In [44]:
df_trips["sourceid"].nunique()

22

In [45]:
df_trips[df_trips["sourceid"].astype(str).isin(lista_agebs)]['sourceid'].nunique()

22

In [46]:
df_trips[df_trips["dstid"].astype(str).isin(lista_agebs)]['sourceid'].nunique()

22

In [47]:
df_trips.dtypes

sourceid                                      int64
dstid                                         int64
hod                                           int64
mean_travel_time                            float64
standard_deviation_travel_time              float64
geometric_mean_travel_time                  float64
geometric_standard_deviation_travel_time    float64
dtype: object

In [13]:
df_trips.reset_index(drop=True, inplace=True)

In [14]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,1093,1426,9,2308.88,464.96,2266.58,1.21
1,1226,1322,0,875.60,946.56,639.82,1.99
2,1534,1430,1,103.86,167.73,74.55,1.91
3,1106,1093,5,424.84,197.97,398.12,1.40
4,1534,1375,4,195.40,209.84,163.43,1.59


### tiempo máximo y mínimos por viaje y hora.
Suponemos una distribución normal con media 'mean_travel_time' y desviación estandar 'standard_deviation_travel_time>'

In [13]:
def travel_min(mean,std):
    min = scipy.stats.norm(mean, std).ppf(.10)
    #max = scipy.stats.norm(mean, std).ppf(.90)
    return(min)
def travel_max(mean,std):
    #min = scipy.stats.norm(mean, std).ppf(.10)
    max = scipy.stats.norm(mean, std).ppf(.90)
    return(max)

In [14]:
df_trips['min_travel_time'] = 0
df_trips['max_travel_time'] = 0

In [15]:
df_trips['min_travel_time'] =  df_trips.apply(
    lambda row : travel_min(row['mean_travel_time'], 
                            row['standard_deviation_travel_time']), axis = 1)

In [16]:
df_trips['max_travel_time'] =  df_trips.apply(
    lambda row : travel_max(row['mean_travel_time'], 
                            row['standard_deviation_travel_time']), axis = 1)

In [17]:
df_trips.to_csv('./data/procesado/mexico_city-agebs-2019-2-All-HourlyAggregate_filter.csv')

In [20]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,min_travel_time,max_travel_time
0,1375,2067,11,2125.00,640.25,2039.71,1.32,1304.486610,2945.513390
1,1534,2452,5,766.98,243.10,743.55,1.28,455.434814,1078.525186
2,1106,1944,19,1298.45,258.22,1274.28,1.21,967.527755,1629.372245
3,1303,1894,16,1124.22,278.44,1098.68,1.22,767.384782,1481.055218
4,1375,1174,16,1222.41,403.44,1173.74,1.31,705.380836,1739.439164


In [22]:
df_trips.groupby(['sourceid','dstid','hod']).mean().reset_index(drop=False)

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,min_travel_time,max_travel_time
0,1002,7,18,1846.57,272.25,1827.18,1.16,1497.667586,2195.472414
1,1002,7,20,1807.17,457.94,1750.76,1.29,1220.296276,2394.043724
2,1002,7,21,1898.33,214.99,1885.78,1.12,1622.809229,2173.850771
3,1002,12,0,774.21,269.38,745.10,1.28,428.985639,1119.434361
4,1002,12,1,738.80,118.28,730.60,1.16,587.218081,890.381919
...,...,...,...,...,...,...,...,...,...
385232,1534,5084,19,2972.79,1158.85,2747.38,1.50,1487.663968,4457.916032
385233,1534,5084,20,2678.27,938.70,2530.81,1.39,1475.277545,3881.262455
385234,1534,5084,21,2593.15,1236.06,2410.00,1.42,1009.075372,4177.224628
385235,1534,5084,22,1894.39,478.01,1838.94,1.27,1281.795536,2506.984464


In [15]:
df_trips['sourceid'].nunique()

22

In [16]:
df_trips['dstid'].nunique()

25